# Question 1


In [1]:
# Import libraries
import sqlite3

# Define database connections
conn = sqlite3.connect("inventory.db")
cursor = conn.cursor()

# Create tables if not already present
cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
  product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  product_name TEXT NOT NULL,
  product_price REAL NOT NULL,
  product_quantity INTEGER NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS sales (
  sale_id INTEGER PRIMARY KEY AUTOINCREMENT,
  sale_date DATE NOT NULL,
  product_name TEXT NOT NULL,
  sale_total REAL NOT NULL
);
""")


class Store:
    def __init__(self):
        self.conn = conn
        self.cursor = cursor

    def add_product(self, name, price, quantity):
        try:
            self.cursor.execute(
                "INSERT INTO products (product_name, product_price, product_quantity) VALUES (?, ?, ?)",
                (name, price, quantity),
            )
            self.conn.commit()
            print("Product added successfully!")
        except sqlite3.Error as e:
            print("Error adding product:", e)

    def remove_product(self, product_id):
        try:
            self.cursor.execute("DELETE FROM products WHERE product_id = ?", (product_id,))
            self.conn.commit()
            print("Product removed successfully!")
        except sqlite3.Error as e:
            print("Error removing product:", e)

    def update_product(self, product_id, name, price, quantity):
        try:
            self.cursor.execute(
                "UPDATE products SET product_name = ?, product_price = ?, product_quantity = ? WHERE product_id = ?",
                (name, price, quantity, product_id),
            )
            self.conn.commit()
            print("Product updated successfully!")
        except sqlite3.Error as e:
            print("Error updating product:", e)

    def display_products(self):
        try:
            self.cursor.execute("SELECT * FROM products")
            rows = self.cursor.fetchall()
            print("-" * 50)
            print(
                f"{'Product ID':<15} {'Name':<25} {'Price':<10} {'Quantity':<10}"
            )
            print("-" * 50)
            for row in rows:
                print(
                    f"{row[0]:<15} {row[1]:<25} {row[2]:<10.2f} {row[3]:<10}"
                )
            print("-" * 50)
        except sqlite3.Error as e:
            print("Error displaying products:", e)

    def sell_product(self, product_id, sale_date, quantity):
        try:
            # Check if product exists and has enough quantity
            self.cursor.execute(
                "SELECT product_quantity FROM products WHERE product_id = ?", (product_id,)
            )
            stock = self.cursor.fetchone()[0]
            if stock < quantity:
                print("Insufficient stock for this product!")
                return
            # Reduce product quantity
            self.cursor.execute(
                "UPDATE products SET product_quantity = product_quantity - ? WHERE product_id = ?",
                (quantity, product_id),
            )
            # Calculate sale total
            sale_total = quantity * (
                self.cursor.execute(
                    "SELECT product_price FROM products WHERE product_id = ?",
                    (product_id,),
                )
                .fetchone()[0]
            )
            # Add sale record
            self.cursor.execute(
                "INSERT INTO sales (sale_date, product_name, sale_total) VALUES (?, ?, ?)",
                (sale_date, self.get_product_name(product_id), sale_total),
            )
            self.conn.commit()
            print(f"Product sold successfully! Total: ${sale_total:.2f}")
        except sqlite3.Error as e:
            print("Error selling product:", e)


In [2]:
def main():
    store = Store()

    while True:
        print("\nInventory Management System")
        print("1. Add Product")
        print("2. Remove Product")
        print("3. Update Product")
        print("4. Display Products")
        print("5. Sell Product")
        print("6. Exit")

        choice = input("\nEnter your choice (1-6): ")

        if choice == "1":
            name = input("Enter product name: ")
            price = float(input("Enter product price: "))
            quantity = int(input("Enter product quantity: "))
            store.add_product(name, price, quantity)
        elif choice == "2":
            product_id = int(input("Enter product ID: "))
            store.remove_product(product_id)
        elif choice == "3":
            product_id = int(input("Enter product ID: "))
            name = input("Enter new product name (optional): ")
            price = input("Enter new product price (optional, leave blank for no change): ")
            quantity = input("Enter new product quantity (optional, leave blank for no change): ")
            if price:
                price = float(price)
            if quantity:
                quantity = int(quantity)
            store.update_product(product_id, name, price, quantity)
        elif choice == "4":
            store.display_products()
        elif choice == "5":
            product_id = int(input("Enter product ID: "))
            sale_date = input("Enter sale date (YYYY-MM-DD): ")
            quantity = int(input("Enter quantity to sell: "))
            store.sell_product(product_id, sale_date, quantity)
        elif choice == "6":
            print("Exiting application...")
            break
        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()



Inventory Management System
1. Add Product
2. Remove Product
3. Update Product
4. Display Products
5. Sell Product
6. Exit

Enter your choice (1-6): 4
--------------------------------------------------
Product ID      Name                      Price      Quantity  
--------------------------------------------------
1               Apple                     2.00       35        
--------------------------------------------------

Inventory Management System
1. Add Product
2. Remove Product
3. Update Product
4. Display Products
5. Sell Product
6. Exit

Enter your choice (1-6): 6
Exiting application...


# Question 2

In [3]:
import tkinter as tk

def calculate_tax():
    #Get the input values from the user
    monthly_salary = float(salary_input.get())
    tax_period = period_var.get()
    
    #Calculate thee annual salary based on the tax period selected
    if tax_period == "monthly":
        annual_salary = monthly_salary * 12
    else:
        annual_salary = monthly_salary

    tax_due = 0
    
    # Calaculate the tax due based on the tax table
    if annual_salary == 0:
        tax_due = 0
    elif annual_salary <= 237100:
        tax_due = (annual_salary) * 0.18
    elif annual_salary <= 370500:
        tax_due = 42678 + (annual_salary - 237100) * 0.26
    elif annual_salary <= 512800:
        tax_due = 77362 + (annual_salary - 370500) * 0.31
    elif annual_salary <= 673000:
        tax_due = 121475 + (annual_salary - 512800) * 0.36
    elif annual_salary <= 857900:
        tax_due = 179147 + (annual_salary - 673000) * 0.39
    elif annual_salary <= 1817000:
        tax_due = 251258 + (annual_salary - 857900) * 0.41
    else:
        tax_due = 644489 + (annual_salary - 1817000) * 0.45

    # Display the tax due 
    if tax_period == "monthly":
        tax_due /= 12

    result_label.config(text=f"Tax due: R {tax_due:.2f}")

# Initialize the main window
root = tk.Tk()
root.title("SA Income Tax Calculator")
root.geometry("300x300")

# Create a label and entry for the monthly salary
tk.Label(root, text="Monthly salary:").grid(row =0 , column = 0, padx =15)
salary_input = tk.Entry(root, width =10)
salary_input.grid(row = 0, column = 1)

# Create a option menu for the tax period
tk.Label(root, text="Tax period:").grid( row =1, column =0)
period_var = tk.StringVar(root)
period_var.set("monthly")

w = tk.OptionMenu(root, period_var, "monthly", "yearly")
w.grid(row=1, column =1)

# Button to calculate the tax and display the result
tk.Button(root, text="Calculate Tax", command=calculate_tax).grid(row=2, column=0, columnspan = 3)

result_label = tk.Label(root, text="")
result_label.grid(row =3, column=0, columnspan =2)

# Start the main event loop
root.mainloop()